In [1]:
from deps.dobot_api import DobotApiDashboard, DobotApiMove
from deps import utils
from deps import cv_core
import numpy as np
import matplotlib.pyplot as plt
import cv2
from pynput import keyboard
import time

In [12]:
def sort_anchors(inp):
    maxx = sorted(np.squeeze(inp), key=lambda x: x[0])

    left = maxx[:2]
    right = maxx[-2:]

    l_sorty = sorted(left, key=lambda x: x[1])
    r_sorty = sorted(right, key=lambda x: x[1])

    return np.vstack((l_sorty, r_sorty))

In [7]:
dash = DobotApiDashboard('192.168.1.6', 29999)
move = DobotApiMove('192.168.1.6', 30003)
keys = utils.Keyboard(dash)

In [13]:
dash.ClearError()
dash.EnableRobot()

'0,{},EnableRobot();'

In [8]:
utils.report_mode(dash)
utils.report_error(dash)

Mode: ROBOT_MODE_ERROR

|CODE: 70|
__________________________________________________
Possible reasons from ALARM SERVO:
Invalid control mode.
__________________________________________________
Possible reasons from ALARM CONTROLLER:
Positive limit alarm of Joint4.


In [53]:
dash.SetCollisionLevel(1)

'0,{},SetCollisionLevel(1);'

In [9]:
dash.DisableRobot()

'0,{},DisableRobot();'

In [55]:
utils.get_pose(dash)

X = 321.505463
Y = 0.0
Z = -34.254505
r = 22.918932


array([321.505463,   0.      , -34.254505,  22.918932])

In [54]:
move.MovL(321.505454, 0, -50, 22.918932)

'0,{},MovL(321.505454,0.000000,-50.000000,22.918932);'

In [14]:
keys.execute()

Position saved!
Position saved!
Position saved!
Position saved!
Special key pressed: Key.esc


In [17]:
cap = cv2.VideoCapture(0)
cap = cv_core.set_res(cap, (1600, 1200))
cv2.namedWindow('frame',  cv2.WINDOW_NORMAL)
cv2.resizeWindow('frame', 1348, 1011)

cameraMatrix = np.load('./cam_matrices/cam_mtx.npy')
dist = np.load('./cam_matrices/dist.npy')
newCameraMatrix = np.load('./cam_matrices/newcam_mtx.npy')

all_anchors = []

idx = 1
while True:
    ret, frame = cap.read()
    frame = cv2.undistort(frame, cameraMatrix, dist, None, newCameraMatrix)
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    detected_circles = cv2.HoughCircles(image=gray,
                                        method=cv2.HOUGH_GRADIENT,
                                        dp=1,
                                        minDist=30,
                                        param1=100,
                                        param2=40,
                                        minRadius=20,
                                        maxRadius=40
                                        )


    if detected_circles is not None:
        plot_img = frame.copy()
        circles = np.uint16(np.around(detected_circles))
        if np.squeeze(circles).shape[0] == 4:
            all_anchors.append(sort_anchors(circles))

        for i in circles[0,:]:
            center = (i[0], i[1])
            cv2.circle(plot_img, center, 1, (255, 0 ,0), 3)
            radius = i[2]
            cv2.circle(plot_img, center, radius, (0, 255 ,0), 3)

        cv2.putText(plot_img, f"Frame {idx}", (25,25), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,255,0), 2)

        # for i in all_anchors[0]:
        #     center = (int(i[0]), int(i[1]))
        #     cv2.circle(plot_img, center, 1, (0, 0 ,255), 3)
        #     radius = i[2]
        #     cv2.circle(plot_img, center, int(radius), (0, 255 ,0), 3)
    idx += 1
    cv2.imshow('frame', plot_img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()

plot_img = frame.copy()
means = np.mean(all_anchors, axis = 0)
for i in means:
    center = (int(i[0]), int(i[1]))
    cv2.circle(plot_img, center, 1, (0, 0 ,255), 3)
    radius = i[2]
    cv2.circle(plot_img, center, int(radius), (0, 0, 255), 3)
cv2.putText(plot_img, "Captured:", (25,25), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,255,0), 2)
cv2.imshow('frame', plot_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:
means

array([[ 291.08695652,  102.52173913,   30.86956522],
       [ 299.36956522, 1129.45652174,   30.06521739],
       [1317.17391304,   99.2173913 ,   32.63043478],
       [1312.08695652, 1128.        ,   31.08695652]])

In [19]:
centers = means[:, :-1]

In [20]:
# print("Detecting Anchors")
# while True:
#     cap = cv2.VideoCapture(0)
#     cap = cv_core.set_res(cap, (1600, 1200))
#     centers = cont.wait_for_anchors(cap, show = True)
#     cap.release()
#     if len(centers) == 4:
#         break


xys = [(arr[0], arr[1]) for arr in keys.coords]
pix_coor = utils.assign_corners(centers)
robot_coor = utils.assign_corners(xys, reverse=True)


features_mm_to_pixels_dict = {}
for key, value in robot_coor.items():
    features_mm_to_pixels_dict[value] = pix_coor[key]


tf_mtx = cv_core.compute_tf_mtx(features_mm_to_pixels_dict)

In [26]:
cont = cv_core.Contours()

def on_change(val): pass

offset = 110
cap = cv2.VideoCapture(0)
cap = cv_core.set_res(cap, cv_core.camera_res_dict['1200'])
#out = cv2.VideoWriter('out.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 25, (640,480))
cont = cv_core.Contours()

cameraMatrix = np.load('./cam_matrices/cam_mtx.npy')
dist = np.load('./cam_matrices/dist.npy')
newCameraMatrix = np.load('./cam_matrices/newcam_mtx.npy')

cv2.namedWindow('frame',  cv2.WINDOW_NORMAL)
cv2.resizeWindow('frame', 1348, 1011)
cv2.createTrackbar('Manual Lock', 'frame', 0, 1, on_change)
cv2.setMouseCallback('frame', cont.mousecallback)

idx = 0
prev_point = (0,0,0)
while(True):
    ret, frame = cap.read()
    frame = cv2.undistort(frame, cameraMatrix, dist, None, newCameraMatrix)
    # frame = np.split(frame,2, axis = 1)[1]
    val = cv2.getTrackbarPos('Manual Lock', 'frame')
    if val == 1:
        cont.locked = True
    else:
        cont.locked = False

    pt = cont.find_contours(frame, 10, offset)
    a,b,r = pt
    plot_img = frame.copy()
    cv2.circle(plot_img, (a, b), r, (0, 255, 0), 2)
    cv2.circle(plot_img, (a, b), 1, (0, 0, 255), 3)
    cv2.circle(plot_img, (a, b), r - offset, (0, 0, 255), 2)
    cv2.drawContours(plot_img, cont.singular, -1,(0,255, 0),2)
    cv2.drawContours(plot_img, cont.clusters, -1,(0,0,255),2)

    if cont.selected:
        cv2.drawContours(plot_img, cont.selected, -1,(255,0,0),2)

    for c in cont.singular:
        # compute the center of the contour
        M = cv2.moments(c)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        # draw the contour and center of the shape on the image
        #cv2.drawContours(plot_img, [c], -1, (0, 255, 0), 2)
        cv2.circle(plot_img, (cX, cY), 2, (0, 0, 255), -1)
        cv2.putText(plot_img, f"{cX},{cY}", (cX - 20, cY - 20),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # cv2.putText(plot_img, f"{cv2.contourArea(c)}", (cX - 20, cY - 20),
        # cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 1)
    #out.write(with_contours)

    for c in cont.clusters:
        # compute the center of the contour
        M = cv2.moments(c)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        # draw the contour and center of the shape on the image
        #cv2.drawContours(plot_img, [c], -1, (0, 255, 0), 2)
        cv2.circle(plot_img, (cX, cY), 2, (0, 0, 255), -1)
        # cv2.putText(plot_img, f"{cX},{cY}", (cX - 20, cY - 20),
        # cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)

        # cv2.putText(plot_img, f"{cv2.contourArea(c)}", (cX - 20, cY - 20),
        # cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 1)


    if prev_point is pt:
        idx += 1
    else:
        idx = 0
    prev_point = pt


    if idx >= 30:
        message = 'LOCKED'
        color = (0,255,0)
    else:
        message = 'SEARCHING'
        color = (0,0,255)

    cv2.putText(plot_img, "TARGET:", (25,25), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,255,0), 2)
    cv2.putText(plot_img, f"{message}", (125,25), cv2.FONT_HERSHEY_SIMPLEX, 0.75, color, 2)
    cv2.putText(plot_img, f"Found: {len(cont.singular)}", (25,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,255,0), 2)

    #cv2.setWindowProperty('frame', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    cv2.imshow('frame',plot_img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [11]:
dash.DisableRobot()

'0,{},DisableRobot();'

In [22]:
dash.ClearError()
dash.EnableRobot()

'0,{},EnableRobot();'

In [28]:
#centers_cuboids = cont.contour_centers(cont.singular)
centers_cuboids = cont.contour_centers(cont.selected)

dash.ClearError()
dash.EnableRobot()

utils.default_pos(move)
for center in centers_cuboids:
    X, Y, _ = tf_mtx @ (center[0], center[1], 1)
    move.MovL(X, Y, -70,  0)
    time.sleep(0.5)
    move.MovL(X, Y, -91,  0)
    time.sleep(0.5)
    move.MovL(X, Y, -70,  0)
    time.sleep(0.5)
utils.default_pos(move)

'0,{},JointMovJ(0.000000,0.000000,0.000000,0.000000);'

In [15]:
keys = utils.Keyboard(dash)
keys.execute()
xys = [(arr[0], arr[1]) for arr in keys.coords]

Position saved!
Position saved!
Position saved!
Position saved!
Special key pressed: Key.esc


In [17]:
robot_coor = utils.assign_corners(xys, reverse=True)

In [18]:
mean_ux = np.mean([robot_coor['ul'][0], robot_coor['ur'][0]])
mean_lx = np.mean([robot_coor['ll'][0], robot_coor['lr'][0]])

mean_uy = np.mean([robot_coor['ur'][1], robot_coor['lr'][1]])
mean_ly = np.mean([robot_coor['ll'][1], robot_coor['ul'][1]])

In [19]:
well_plate_xs = np.linspace(mean_ux, mean_lx, 12)
well_plate_ys = np.linspace(mean_uy, mean_ly, 8)

In [20]:
dash.ClearError()
dash.EnableRobot()

utils.default_pos(move)
for x in well_plate_xs:
    for y in well_plate_ys:
        move.MovL(x, y, -70,  0)
        time.sleep(0.15)
        move.MovL(x, y, -80,  0)
        time.sleep(0.15)
        move.MovL(x, y, -70,  0)
        time.sleep(0.15)

utils.default_pos(move)

'0,{},JointMovJ(0.000000,0.000000,0.000000,0.000000);'

In [44]:
well_plate_coords = np.concatenate(np.stack(np.meshgrid(well_plate_xs, well_plate_ys), axis = 2))

In [55]:
dash.ClearError()
dash.EnableRobot()

utils.default_pos(move)
for idx, center in enumerate(centers_cuboids):
    X, Y, _ = tf_mtx @ (center[0], center[1], 1)
    move.MovL(X, Y, -70,  0)
    time.sleep(0.5)
    move.MovL(X, Y, -95,  0)
    time.sleep(0.5)
    move.MovL(X, Y, -70,  0)
    time.sleep(0.5)
    
    x,y = well_plate_coords[idx]
    move.MovL(x, y, -70,  0)
    time.sleep(0.5)
    move.MovL(x, y, -90,  0)
    time.sleep(0.5)
    move.MovL(x, y, -70,  0)
    time.sleep(0.5)

utils.default_pos(move)

'0,{},JointMovJ(0.000000,0.000000,0.000000,0.000000);'